In [1]:
import os
from docx import Document
from docx.shared import Inches
from datetime import datetime
import uuid

In [2]:
DOCUMENTOS_PATH = "documentos/"
FIRMAS_PATH = "firmas/"
MACHOTES_PATH = "machotes/"

In [3]:
def rellenar_campos_docx(path_machote, datos, id_documento):
    """
    Rellena los campos {campo} en un documento .docx y guarda usando un ID único.

    Args:
        path_machote (str): Ruta del documento machote (.docx) a usar.
        datos (dict): Diccionario con claves {campo} y sus valores correspondientes.
        id_documento (str or int): ID único del documento (ejemplo: 123456)
    """
    # Cargar el documento
    doc = Document(path_machote)

    # Reemplazar en cada párrafo
    for p in doc.paragraphs:
        for key, value in datos.items():
            if key in p.text:
                p.text = p.text.replace(key, str(value))

    # Reemplazar dentro de tablas también
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for key, value in datos.items():
                    if key in cell.text:
                        cell.text = cell.text.replace(key, str(value))

    # Definir ruta de salida basada en el ID
    path_salida = os.path.join(DOCUMENTOS_PATH, f"doc_{id_documento}.docx")

    # Guardar el documento rellenado en su nueva ruta
    doc.save(path_salida)
    return path_salida

In [4]:
def crear_documento_donacion(datos):
    """
    Crea todo el flujo de generación del documento de donación.

    Args:
        path_machote (str): Ruta al machote de especie o dinero.
        datos (dict): Diccionario de datos a rellenar.

    Returns:
        dict: Contiene el ID del documento, tipo de donación y fecha de donación.
    """
    # 1. Generar ID
    id_documento = uuid.uuid4()
    
     # 2. Determinar tipo de donación
    if "{monto_donado}" in datos or "monto_donado" in datos:
        tipo_donacion = "dinero"
        path_machote = os.path.join(MACHOTES_PATH, "Donacion_Dinero.docx")
    elif "{lista_articulos}" in datos or "lista_articulos" in datos:
        tipo_donacion = "especie"
        path_machote = os.path.join(MACHOTES_PATH, "Donacion_Especie.docx")
    else:
        tipo_donacion = "desconocido"  # En caso de error
        
    


    # 3. Determinar la ruta del machote
    path_final = rellenar_campos_docx(path_machote, datos, id_documento)

   

    # 4. Obtener fecha
    fecha_donacion = datos.get("{fecha_donacion}", datos.get("fecha_donacion", "fecha_desconocida"))

    return {
        "id_documento": id_documento,
        "tipo_donacion": tipo_donacion,
        "fecha_donacion": fecha_donacion,
        "path_documento": path_final
    }

In [5]:
today = datetime.now()
fecha_hoy = today.strftime("%d de %B de %Y")
datos_donacion = {
    "{fecha_donacion}": fecha_hoy,
    "{nombre_donante}": "Juan Pérez",
    "{lista_articulos}": "- 10 paquetes de arroz\n- 5 cobijas\n- 3 cajas de medicamento"
}

resultado = crear_documento_donacion(
    datos=datos_donacion
)

print(resultado)


{'id_documento': UUID('6993f3dc-f699-4cf0-a598-7f908a973257'), 'tipo_donacion': 'especie', 'fecha_donacion': '13 de May de 2025', 'path_documento': 'documentos/doc_6993f3dc-f699-4cf0-a598-7f908a973257.docx'}
